In [1]:
import pyhopper
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import random
import time
import os

# Set global seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

# Load and preprocess data
df = pd.read_csv("ECTA 2023 Dataset.csv")
features = ['Relative Humidity', 'Wind Speed', 'Visibility', 'Pressure', 'Wind_Chill', 'Dewpoint Temp']
target = 'Temperature'

df.dropna(inplace=True)
X = df[features].values
Y = df[target].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=SEED)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

top_maes = []

# Define the objective function
def objective_function(params):
    lr = params['lr']
    n1 = int(params['n1'])
    n2 = int(params['n2'])
    bs = int(params['bs'])

    model = keras.Sequential([
        keras.layers.Input(shape=(X_train.shape[1],)),
        keras.layers.Dense(n1, activation='relu'),
        keras.layers.Dense(n2, activation='relu'),
        keras.layers.Dense(1)
    ])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
                  loss='mse', metrics=['mae'])
    model.fit(X_train, Y_train, epochs=10, batch_size=bs, verbose=0)
    loss, mae = model.evaluate(X_test, Y_test, verbose=0)

    top_maes.append(mae)
    print(f"MAE: {mae:.4f} | Params: {params}")
    return mae

# Define the search space
search_space = {
    'lr': pyhopper.float(0.0005, 0.05),
    'n1': pyhopper.int(10, 100),
    'n2': pyhopper.int(10, 100),
    'bs': pyhopper.int(16, 128)
}

# Initialize and run the search
search = pyhopper.Search(search_space)
best_params = search.run(
    objective_function,
    direction="minimize",
    runtime="10m",
    n_jobs=1
)

print("\n Best configuration found:")
print(best_params)
final_mae = objective_function(best_params)
print(f"Final MAE: {final_mae:.4f}")

# Show top MAEs during the search
top_maes.sort()
print("\n Top 5 lowest MAEs found during search:")
for i, val in enumerate(top_maes[:5]):
    print(f"{i+1}. MAE = {val:.4f}")


  0%|          | [00:00<?]

Search is scheduled for 10:00 (m:s)
MAE: 0.1873 | Params: {'lr': 0.02525, 'n1': 55, 'n2': 55, 'bs': 72}
MAE: 0.1045 | Params: {'lr': 0.01930694305205854, 'n1': 91, 'n2': 28, 'bs': 23}
MAE: 0.1012 | Params: {'lr': 0.036561010451384, 'n1': 100, 'n2': 10, 'bs': 18}
MAE: 0.0921 | Params: {'lr': 0.034826485637454566, 'n1': 100, 'n2': 18, 'bs': 18}
MAE: 0.0869 | Params: {'lr': 0.025938616014907302, 'n1': 100, 'n2': 31, 'bs': 18}
MAE: 0.0936 | Params: {'lr': 0.02942714476058354, 'n1': 100, 'n2': 37, 'bs': 18}
==================== Summary ====================
Mode              : Best f  : Steps : Time       
----------------  : ----    : ----  : ----       
Initial solution  : 0.1873  : 1     : 48 s       
Random seeding    : 0.1012  : 2     : 03:20 (m:s)
Local sampling    : 0.08692 : 3     : 05:04 (m:s)
----------------  : ----    : ----  : ----       
Total             : 0.08692 : 6     : 09:12 (m:s)

 Best configuration found:
{'lr': 0.025938616014907302, 'n1': 100, 'n2': 31, 'bs': 18}
MAE: